Final Project

In [1]:
import plotly.express as px
import pycountry
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd


In [29]:
df = pd.read_csv('CO2_Emissions_1960-2018.csv')

# reshape the date to be able to display co2 emmsion per years
df = df.melt(id_vars=["Country Name"], var_name="Year", value_name="Emission Per Year")

In [30]:

# get gdp
df_gdp = pd.read_csv('countries_gdp_hist.csv', sep=';')
df['Year'] = df['Year'].astype(int)
df_gdp['year'] = df_gdp['year'].astype(int)



# convert country names to iso format
def get_iso_alpha_3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

df['iso_alpha'] = df['Country Name'].apply(get_iso_alpha_3)


df_merged = df.merge(df_gdp, left_on=['iso_alpha', 'Year'], right_on=['country_name', 'year'], how='outer')

# init dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    # create worldmap
    dcc.Graph(
        id='choropleth-map',
        figure=px.choropleth(
            df_merged,
            locations="iso_alpha",
            color="Emission Per Year",
            scope="world",
            animation_frame="Year",
            hover_name="Country Name",
            width=1000,
            height=600
        )
    ),
    dcc.Graph(id='bar-chart')
])

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('choropleth-map', 'clickData')]
)

## prepare bar chart after cllick on country
def update_bar_chart(clickData):
    if clickData is None:
        country_name = 'Switzerland'
    else:
        country_name = clickData['points'][0]['hovertext']
    filtered_df = df_merged[df_merged['country_name'] == country_name]
    fig = px.bar(filtered_df, x='year', y='total_gdp_million', title=f'GDP for country: {country_name}')
    return fig

# def update_bar_chart(clickData):
#     if clickData is None:
#         country_name = 'Switzerland'
#     else:
#         country_name = clickData['points'][0]['hovertext']
    
#     # Filtere die Daten für das angeklickte Land
#     filtered_df = df_merged[df_merged['country_name'] == country_name]
    
#     # Erstelle ein Bar-Chart
#     fig = px.bar(filtered_df, x='year', y='total_gdp_million', title=f'GDP for country: {country_name}')
    
#     # Füge eine rote Linie hinzu, die 'Emission Per Year' darstellt
#     if not filtered_df.empty:
#         fig.add_shape(
#             type="line",
#             x0=filtered_df['year'].min(),
#             y0=filtered_df['Emission Per Year'].mean(),  # Beispiel: Durchschnitt
#             x1=filtered_df['year'].max(),
#             y1=filtered_df['Emission Per Year'].mean(),
#             line=dict(color="red", width=2, dash="dash"),
#         )
    
#     # Aktualisiere die Achsentitel für mehr Klarheit
#     fig.update_layout(
#         xaxis_title="Year",
#         yaxis_title="GDP (in million)",
#     )
    
#     return fig


if __name__ == '__main__':
    app.run_server(debug=True)


In [14]:
def update_bar_chart(clickData):
    if clickData is None:
        country_name = 'Switzerland'
    else:
        country_name = clickData['points'][0]['hovertext']
    
    # Filtere die Daten für das angeklickte Land
    filtered_df = df_merged[df_merged['country_name'] == country_name]
    
    # Erstelle ein Bar-Chart
    fig = px.bar(filtered_df, x='year', y='total_gdp_million', title=f'GDP for country: {country_name}')
    
    # Füge eine rote Linie hinzu, die 'Emission Per Year' darstellt
    if not filtered_df.empty:
        fig.add_shape(
            type="line",
            x0=filtered_df['year'].min(),
            y0=filtered_df['Emission Per Year'].mean(),  # Beispiel: Durchschnitt
            x1=filtered_df['year'].max(),
            y1=filtered_df['Emission Per Year'].mean(),
            line=dict(color="red", width=2, dash="dash"),
        )
    
    # Aktualisiere die Achsentitel für mehr Klarheit
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="GDP (in million)",
    )
    
    return fig
